In [ ]:
import sklearn
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Este é o dicionário de dados para o dataset do Projeto 1:
1. **ID**: Identificador único do cliente.
2. **Ano_Nascimento**: Ano de nascimento do cliente.
3. **Educacao**: Nível de educação do cliente (ex.: Graduação, PhD).
4. **Estado_Civil**: Estado civil do cliente (ex.: Casado, Solteiro).
5. **Renda**: Renda anual do cliente em reais.
6. **Criancas_Em_Casa**: Número de crianças pequenas na casa do cliente.
7. **Adolescentes_Em_Casa**: Número de adolescentes na casa do cliente.
8. **Data_Cadastro_Cliente**: Data em que o cliente foi cadastrado na empresa.
9. **Gasto_Vinhos**: Total gasto em vinhos no último ano.
10. **Gasto_Frutas**: Total gasto em frutas no último ano.
11. **Gasto_Carnes**: Total gasto em carnes no último ano.
12. **Gasto_Peixes**: Total gasto em peixes no último ano.
13. **Gasto_Doces**: Total gasto em doces no último ano.
14. **Gasto_Outros**: Total gasto em outros produtos no último ano.
15. **Num_Compras_Desconto**: Número de compras feitas com desconto.
16. **Num_Compras_Web**: Número de compras feitas através da web.
17. **Num_Compras_Catalogo**: Número de compras feitas usando um catálogo.
18. **Num_Compras_Loja**: Número de compras feitas diretamente na loja.
19. **Num_Visitas_Web_Mes**: Número de visitas ao site da empresa por mês.
20. **Aceitou_Campanha_1**: Indica se o cliente aceitou a oferta na primeira campanha (0 = Não, 1 = Sim)
21. **Aceitou_Campanha_2**: Indica se o cliente aceitou a oferta na segunda campanha (0 = Não, 1 = Sim).
22. **Aceitou_Campanha_3**: Indica se o cliente aceitou a oferta na terceira campanha (0 = Não, 1 = Sim).
23. **Aceitou_Campanha_4**: Indica se o cliente aceitou a oferta na quarta campanha (0 = Não, 1 = Sim).
24. **Aceitou_Campanha_5**: Indica se o cliente aceitou a oferta na quinta campanha (0 = Não, 1 = Sim).
25. **Aceitou_Campanha_6**: Indica se o cliente aceitou a oferta na sexta campanha (0 = Não, 1 = Sim)



In [ ]:
archive = 'data/dataset.csv'

df = pd.read_csv(archive)
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

## Limpeza e processamento inicial dos dados

In [ ]:
df.columns = df.columns.str.replace(' ', '') 
df.columns = [x.lower() for x in df.columns]

df['renda'] = df['renda'].replace('[\$,]', '', regex=True).astype(float)
df['data_cadastro_cliente'] = pd.to_datetime(df['data_cadastro_cliente']) 

# df['aceitou_campanha_1'] = df['aceitou_campanha_1'].astype('category')
# df['aceitou_campanha_2'] = df['aceitou_campanha_2'].astype('category')
# df['aceitou_campanha_3'] = df['aceitou_campanha_3'].astype('category')
# df['aceitou_campanha_4'] = df['aceitou_campanha_4'].astype('category')
# df['aceitou_campanha_5'] = df['aceitou_campanha_5'].astype('category')
# df['aceitou_campanha_6'] = df['aceitou_campanha_6'].astype('category')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(df['renda'])
plt.title('Boxplot da Renda')
plt.xlabel('Renda')
plt.ylabel('Renda')
plt.show()

In [ ]:
Q1 = df['renda'].quantile(0.25)
Q3 = df['renda'].quantile(0.75)
IQR = Q3 - Q1


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f'O valor {lower_bound} está abaixo do padrão')
print(f'O valor {upper_bound} está acima do padrão')

In [ ]:
plt.figure(figsize = (10, 6)) 
sns.displot(df['renda'], color = 'brown', kde=True)  
plt.title('Distribuição de Renda', size = 16)  
plt.xlabel('Renda')
plt.ylabel('Contagem')
plt.show()

Existem várias estratégias para tratar outliers em um conjunto de dados e a escolha da melhor abordagem depende do contexto específico e do objetivo da análise. Aqui estão algumas opções comuns:

**Remoção**: Simplesmente remover os outliers do conjunto de dados. Esta é uma abordagem direta, mas pode não ser ideal se o número de outliers for significativo ou se esses pontos contiverem informações importantes.

**Substituição**: Substituir os valores dos outliers por outros mais representativos, como a média ou mediana dos dados. Esta abordagem é útil se os outliers forem considerados erros de medição ou entrada de dados.

**Transformação**: Aplicar uma transformação aos dados pode reduzir o impacto dos outliers. Transformações comuns incluem logarítmica, raiz quadrada ou Box-Cox.

**Capping (Limitação)**: Definir um limite superior e/ou inferior para os valores dos dados. Valores além desses limites são trazidos para o limite mais próximo. Por exemplo, todos os valores acima do limite superior são definidos para esse limite.

**Análise Separada**: Às vezes, os outliers são mantidos no conjunto de dados e analisados separadamente para entender melhor suas características.

**Análise de Causa**: Investigar a causa dos outliers. Se eles são resultado de um fenômeno real, pode ser importante incluí-los na análise.

A escolha do método depende da natureza dos dados e do objetivo da análise. Em muitos casos, uma combinação dessas estratégias pode ser a mais eficaz. 

In [ ]:
df = df[(df['renda'] >= lower_bound) & (df['renda'] <= upper_bound)]

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(df['renda'])
plt.title('Boxplot da Renda')
plt.xlabel('Renda')
plt.ylabel('Renda')
plt.show()

In [ ]:
plt.figure(figsize = (10, 6)) 
sns.histplot(df['renda'], color = 'brown', kde=True)  
plt.title('Distribuição de Renda', size = 16)  
plt.xlabel('Renda')
plt.ylabel('Contagem')
plt.show()

In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[['renda']])

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df_scaled)

df['renda'] = scaler.inverse_transform(df_imputed)
print(f'Total de valores Ausentes: {df.isnull().sum().sum()}')

## Análise Exploratória e Engenharia de Atributos

In [ ]:
data_boxplot = df.drop(columns = ['id',
                                  'educacao',
                                  'estado_civil',
                                  'criancas_em_casa',
                                  'data_cadastro_cliente',
                                  'adolescentes_em_casa', 
                                  'aceitou_campanha_1', 
                                  'aceitou_campanha_2',
                                  'aceitou_campanha_3', 
                                  'aceitou_campanha_4', 
                                  'aceitou_campanha_5',
                                  'aceitou_campanha_6'])

data_boxplot.plot(subplots = True, layout = (4,5), kind = 'box', figsize = (12,14), patch_artist = True)
plt.subplots_adjust(wspace=0.5);

> Vamos transformar algumas variáveis, criar novas e combinar outras em um trabalho de engenharia de atributos que vai nos ajudar na exploração dos dados.

In [ ]:
ano_atual = datetime.now().year
df['idade'] = ano_atual - df['ano_nascimento']
df['dias_como_cliente'] = (df['data_cadastro_cliente'].max() - df['data_cadastro_cliente']).dt.days
df['total_compras'] = df['num_compras_web'] + df['num_compras_catalogo'] + df['num_compras_loja']
df['gasto_total'] = df.filter(like = 'gasto').sum(axis=1)
df['aceite_campanha'] = df.filter(like = 'aceitou').sum(axis = 1)
df['resposta_campanha'] = df['aceite_campanha'].apply(lambda x: 'Aceitou' if x > 0 else 'Não Aceitou')
df.head()

In [ ]:
df = df.drop(['ano_nascimento', 'data_cadastro_cliente', 'aceite_campanha'], axis = 1)